In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('allsales.csv')  # from january 2016

# remove 'Customer Total'
data = data[data['Year'] != 'Customer Total:']

In [ ]:
def acc_num(row: str) -> str:
    """Retrieve account number from .Prd column.
    A function to get customer number from the .Prd column.
    :param row: a string representing a column name
    :precondition: row is a string representing dataset name
    :postcondition: when iterating through 'row' column, it returns the customer account number from the .Prd column if the
                    data value is 'Customer Number'. Otherwise, it returns a NaN value later used for querying.
    :postcondition: dataset is changed
    :return: account number
    """
    if row['Year'] == 'Customer Number:':
        return row['Prd.']
    else:
        return np.NaN

# apply function to dataset
data['acc_num'] = data.apply(lambda row: acc_num(row), axis=1)

In [ ]:
def acc_name(row: str) -> int:
   """Retrieve account name from .Prd column.
   A function to get customer name from the .Prd column.
   :param row: a string representing a column name
   :precondition: row is a string representing dataset name
   :postcondition: when iterating through 'row' column, it returns the customer account number from the .Prd column if the
                   data value is 'Customer Number'. Otherwise, it returns a NaN value later used for querying.
   :postcondition: dataset is changed
   :return: account name
    """
    if row['Year'] == 'Customer Number:':
        return row['Date']
    else:
        return np.NaN


data['Acc_name'] = data.apply(lambda row: acc_name(row), axis=1)

In [ ]:
def prod_name(row: str) -> str:
    """Retrieve product number from .Prd column.
   A function to get product number from the .Prd column. This works on the fact that if the row value's fourth character
   is a '/', then value in '.Prd' must be a product name
   :param row: a string representing a column name
   :precondition: row is a string representing dataset name
   :postcondition: dataset is changed
   :return: product name
   """
    if str(row['Year'])[3:4] == '/':
        return row['Prd.']
    else:
        return np.NaN


data['Product_name'] = data.apply(lambda row: prod_name(row), axis=1)

In [ ]:
def prod_id(row):
    """Retrieve product number from .Prd column.
   A function to get product number from the .Prd column. This works on the fact that if the row value's fourth character
   is a '/', then value in 'Year' must be a product name
   :param row: a string representing a column name
   :precondition: row is a string representing dataset name
   :postcondition: dataset is changed
   :return: product id
    """
    if str(row['Year'])[3:4] == '/':
        return row['Year']
    else:
        return np.NaN


data['Product_num'] = data.apply(lambda row: prod_id(row), axis=1)

In [ ]:
# forward fill with na values
data.iloc[:, [-1, -2, -3, -4]] = data.iloc[:, [-1, -2, -3, -4]].fillna(method='ffill')

trial = data[
    (data['Year'] == '2020') | (data['Year'] == '2019') | (data['Year'] == '2018') | (data['Year'] == '2017') | (
                data['Year'] == '2016')]
trial = trial.drop(['Year', 'Prd.', 'Type', 'Cost of Sales', 'Percent'], axis=1)


In [ ]:
def credits(row: str) -> str:
    """Identify invoices and credits.
    This function works on the fact that if the first 2 characters in 'Transaction Number' are 'CN', then it definitely
    is a Credit Note. Otherwise, it's definitely and invoice.
    :param row: a string
    :precondition: row is a string representing dataset name
    :postcondition: dataset is changed
    :return: a string
    """
    if str(row['Transaction Number'])[0:2] == 'CN':
        return 'credit'
    else:
        return 'invoice'


trial['type'] = trial.apply(lambda row: credits(row), axis=1)

In [ ]:
trial = trial[trial['type'] != 'credit']  # remove all credit notes, customers might have bought wrong combination
trial = trial[trial['Quantity'].notnull()]  # remove quantity with null value

products = trial[['Transaction Number', 'Product_num', 'Date']]

products = products.drop_duplicates()
# products = products.groupby('Transaction Number')



In [ ]:
import datetime

products['Date'] = pd.to_datetime(products['Date'])
products['Year'] = products['Date'].map(lambda i: i.strftime("%Y"))

# separate transactions into different years
# pro_2016 = products[products['Year']  == '2016']      #year 2016
# pro_2017 = products[products['Year']  == '2017']           #year 2017
# pro_2018 = products[products['Year']  == '2018']       #year 2018


pro_2K19_20 = products[
    (products['Year'] == '2019') | (products['Year'] == '2020')].  # 2019 and 2020 combined for relevancy

freq_items = pro_2K19_20.groupby('Product_num').size().reset_index()
freq_items = freq_items.sort_values(by=0, ascending=False)